In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sqlalchemy import create_engine

In [63]:
os.chdir('./')

# Review nutrient table and get only important values for us

In [99]:
nut = pd.read_csv("./Health_Nutritions _databases/nutrient.csv", usecols=['id', 'name', 'unit_name'])

In [100]:
nut_select = ['Protein', 'Energy', 'Carbohydrate, by difference', 'Sugars, Total NLEA', 'Fatty acids, total saturated', 'Fatty acids, total monounsaturated', 'Fatty acids, total polyunsaturated']
importnut = nut[nut['name'].isin(nut_select)]
importnut #Tabel of nutients, that will be considered

,id,name,unit_name
1,1003,Protein,G
3,1005,"Carbohydrate, by difference",G
5,1008,Energy,KCAL
23,1062,Energy,kJ
24,1063,"Sugars, Total NLEA",G
136,1258,"Fatty acids, total saturated",G
166,1292,"Fatty acids, total monounsaturated",G
167,1293,"Fatty acids, total polyunsaturated",G


In [101]:
nut_list = list(importnut['id'])
nut_list #prepared list used later for narrowing spectrum 

[1003, 1005, 1008, 1062, 1063, 1258, 1292, 1293]

# Review food nutrient table

In [102]:
food_nut = pd.read_csv("./Health_Nutritions _databases/food_nutrient.csv", low_memory=False,
                      usecols=['fdc_id', 'nutrient_id','amount'])

In [68]:
food_nut = food_nut[food_nut['nutrient_id'].isin(nut_list)]

# Review food table

In [70]:
data = pd.read_csv("./Health_Nutritions _databases/food.csv", low_memory=False,
                  usecols=['fdc_id', 'data_type', 'description'], index_col='fdc_id')
data

,data_type,description
fdc_id,,
1105898,experimental_food,Metabolizable Energy of Almonds
1105899,experimental_food,"Metabolizable Energy of Almonds, Food Processi..."
1105900,experimental_food,Metabolizable Energy of Cashews
1105901,experimental_food,Metabolizable Energy of Pistachios
1105902,experimental_food,Metabolizable Energy of Walnuts
...,...,...
1105893,sub_sample_food,"APPLES, FUJI"
1105894,sub_sample_food,"APPLES, FUJI"
1105895,sub_sample_food,"APPLES, FUJI"


##### Import excel sheet to import your own products

own_data = pd.DataFrame({"Name":{}, "Protein":{}, "Carbs":{}, "Sugar":{}, "F. Saturared":{}, "F. Monoun-":{},
                         "F. Poliun-":{}, "Kcal":{}})
own_data

In [98]:
search_word = input("Insert searching product: ")
search_list = data.where(data['description'].str.contains(search_word)).dropna(how='all') 
              #and own_data.where(own_data['Name'].str.contains(search_word)).dropna(how='all')
                
search_list

Insert searching product: Egg, whole


,data_type,description
fdc_id,,
170893,sr_legacy_food,"Egg, whole, raw, frozen, salted, pasteurized"
171287,sr_legacy_food,"Egg, whole, raw, fresh"
172185,sr_legacy_food,"Egg, whole, cooked, omelet"
172186,sr_legacy_food,"Egg, whole, cooked, poached"
172187,sr_legacy_food,"Egg, whole, cooked, scrambled"
172188,sr_legacy_food,"Egg, whole, dried"
172202,sr_legacy_food,"Egg, whole, raw, frozen, pasteurized (Includes..."
173423,sr_legacy_food,"Egg, whole, cooked, fried"
173424,sr_legacy_food,"Egg, whole, cooked, hard-boiled"


# Return nutrient values for choose product (fdc_id)

In [77]:
fdc = int(input("Insert fdc_id of product: "))
work = food_nut.where(food_nut['fdc_id'] == fdc).dropna().rename(columns = {"nutrient_id": "id"})
work

Insert fdc_id of product: 172186


,fdc_id,id,amount
391494,172186.0,1008.0,143.000
391495,172186.0,1062.0,597.000
391522,172186.0,1292.0,3.643
391523,172186.0,1293.0,1.904
391536,172186.0,1005.0,0.710
391543,172186.0,1003.0,12.510
391566,172186.0,1258.0,3.113


# Display how much nutritions for selected portion

In [78]:
weight = float(input("How much of the selected product [g]: "))
result = pd.merge(importnut, work)[["amount", "name", "unit_name"]]
result["amount"] = result["amount"] * weight/100
result

How much of the selected product [g]: 180


,amount,name,unit_name
0,22.5180,Protein,G
1,1.2780,"Carbohydrate, by difference",G
2,257.4000,Energy,KCAL
3,1074.6000,Energy,kJ
4,5.6034,"Fatty acids, total saturated",G
5,6.5574,"Fatty acids, total monounsaturated",G
6,3.4272,"Fatty acids, total polyunsaturated",G


# Selected result sub from our daily needs

In [79]:
protein = float(result["amount"].where(result["name"] == "Protein").dropna())
protein

22.518

In [90]:
parameters = {"Protein": 150, "Carbs": 100, "Sugar": 0, "F. Saturated": 30, "F. Monoun-": 30, "F. Poliun-": 30 }
makro = pd.Series(parameters)
name = nut_select[:1] + nut_select[2:]
result["amount"].where(result["name"].isin(name)).dropna()

0    22.5180
1     1.2780
4     5.6034
5     6.5574
6     3.4272
Name: amount, dtype: float64